In [1]:
import numpy as np
import pandas as pd

## Import Data

In [2]:
def pretty_print_review_and_label(i):
    print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

g = open('reviews.txt','r') # What we know!
reviews = list(map(lambda x:x[:-1],g.readlines()))
g.close()

g = open('labels.txt','r') # What we WANT to know!
labels = list(map(lambda x:x[:-1].upper(),g.readlines()))
g.close()

In [3]:
print('There are {} reviews in this dataset.'.format(len(reviews)))

There are 25000 reviews in this dataset.


In [78]:
reviews[11]

'  it appears that many critics find the idea of a woody allen drama unpalatable .  and for good reason they are unbearably wooden and pretentious imitations of bergman . and let  s not kid ourselves critics were mostly supportive of allen  s bergman pretensions  allen  s whining accusations to the contrary notwithstanding . what i don  t get is this why was allen generally applauded for his originality in imitating bergman  but the contemporaneous brian depalma was excoriated for  ripping off  hitchcock in his suspense  horror films  in robin wood  s view  it  s a strange form of cultural snobbery . i would have to agree with that .  '

In [77]:
labels[11]

'NEGATIVE'

In [8]:
reviews = pd.Series(reviews)
labels = pd.Series(labels)

df = pd.concat([reviews, labels],axis=1)
df.columns = ['reviews','labels']
df.head()

,reviews,labels
0,bromwell high is a cartoon comedy . it ran at ...,POSITIVE
1,story of a man who has unnatural feelings for ...,NEGATIVE
2,homelessness or houselessness as george carli...,POSITIVE
3,airport starts as a brand new luxury pla...,NEGATIVE
4,brilliant over acting by lesley ann warren . ...,POSITIVE


## Preprocess Data

In [9]:
# Denote the features and target
X = df['reviews']
y = df['labels']


## Create Pipeline to Train a Model

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [16]:
from sklearn.neural_network import MLPClassifier

pipe = Pipeline([
    ('vectorize', TfidfVectorizer(min_df = 2, stop_words = 'english', ngram_range=(1,2))),
    ('svd', TruncatedSVD(n_components=300)),
    ('pred_model', MLPClassifier())
])
params= {
    'svd__n_components': [300]
}
gs = GridSearchCV(pipe, param_grid=params)

In [23]:
gs.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vectorize', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=2,
        ngram_range=(1, 2), norm=u'l2', preprocessor=None, smooth_idf...=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'svd__n_components': [300]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [27]:
print('------- Scoring Summary --------')
print('Train Accuracy: {}%'.format(gs.score(X_train,y_train)*100))
print('Test Accuracy: {}%'.format(gs.score(X_test,y_test)*100))

------- Scoring Summary --------
Train Accuracy: 98.5432835821%
Test Accuracy: 86.6666666667%


## Save Trained Model

In [33]:
from sklearn.externals import joblib
save_path = 'gs.pkl'
joblib.dump(gs, save_path)

['gs.pkl']

In [34]:
loaded_gs = joblib.load(save_path)

In [41]:
type(X[0:2])

pandas.core.series.Series

## Load Model and Make Prediction

In [67]:
review1 = """
Over 163 stylish minutes, “Blade Runner 2049” wrestles with nothing less than what it means to be human, serving as a beautiful thematic companion to Ridley Scott’s “Blade Runner,” a film that redefined a genre. It’s too soon to tell if the follow-up will have the influence and staying power of the groundbreaking original but it’s clear from the beginning that this no mere piece of nostalgic fan service. Unlike a lot of reboots or long-delayed sequels that merely remix the themes and characters of the beloved original to give viewers the hollow comfort of familiarity, Denis Villeneuve and his team are remarkably ambitious, using the topics raised by “Blade Runner” to continue the conversation instead of just repeating it to make a buck. To that end, they have made one of the most deeply philosophical and challenging sci-fi films of all time, a movie that never holds your hand as it spirals the viewer through its gorgeous funhouse of the human soul.

Here’s where things get a little tricky for a film critic. Villeneuve and the team at Warner Brothers have asked critics to be incredibly precious with spoilers, not even revealing things that the film does mere minutes into its running time (and even a couple things I believe the trailers give away, but whatever). The way the film reveals its secrets, themes, and connections is one of its greatest strengths, so I’ll heed that directorial request, although I suspect some of the best writing about this film will be done when its themes can be discussed beat by beat and explicitly.

Until then, I’ll just give you the very basics, as beautiful as they are. Riding a wave of gorgeous visuals from the legendary Roger Deakins and a crack effects team, Villeneuve brings us to Los Angeles in 2049. It’s been decades since the action of the first film, but the replicant-destroying Blade Runner is still a profession, personified here by Ryan Gosling as an agent known as K. His duty now is to track down old replicants who have gone into hiding, living long past their originally-programmed lifespan. In the film’s opening, Officer K tracks down a replicant who is just trying to live a peaceful life as a farmer (a spectacled Dave Bautista, doing a great deal with a small role). What he finds there will start what is essentially a detective story, spurring K to solve a mystery about his own past, the history of replicants, the power of memory, and what it means to be a human being. Robin Wright, Jared Leto, and Harrison Ford co-star.

From the minute footage of “Blade Runner 2049” started to leak, it was clear that the director of “Prisoners” and “Arrival” had created a film with a confident, strong visual language. The Oscar talk for the always-an-Academy-bridesmaid Roger Deakins started with the first trailer. The film is undeniably gorgeous, the kind of work that could be appreciated with the volume turned all the way down. Not only are Deakins and Villeneuve great at the “futuristic” aspects of their vision, but they’ve made a film whose most striking imagery often relates to nature. When I think of “2049,” I think of waves crashing, snow falling, and, of course, rain pelting down—an iconic image from the first film that’s almost subverted here in the second half. And Villeneuve and Deakins are often playful within this visually striking world, capturing images that work thematically—I think of a “small” K against gigantic statues at a point when he’s questioning his place in the world or a moment with a hologram that comes off a billboard to remind him of what he’s lost in a fashion that’s ten stories tall—while never losing sight of the pure beauty of it all. It’s one of the most stunningly shot films of not just this year, but the last several. I can’t wait to just see it again, just to bask in its visuals without trying to follow its plot. And the sound design is so remarkable that it’s almost overwhelming—this is a film you don’t passively watch, you experience it.

Don’t worry—this is no hollow visual spectacle. It’s grounded at the same time. As he did with Amy Adams in “Arrival” and Benicio Del Toro in “Sicario,” Villeneuve proves his skill with performance as well, particularly with Ryan Gosling, who gives one of the best performances of his career. Gosling is perfect for this part as he’s always had a vulnerability underneath the handsome façade, and he allows fear and confusion to become operating forces on K’s arc without ever overselling the deep emotion of the piece. It’s a fantastic performance, and Villeneuve draws great ones from Sylvia Hoeks and Ana de Armas as well. The film did drag a little for me near the end of the first hour when I wanted it to pick up the pace, and some of the characters feel like they do things dictated by plot necessity more than believable behavior, but that's a criticism that could fade on repeat viewing.

It would have been incredibly easy to reboot “Blade Runner” directly, merely continuing Deckard and Rachel’s story from the first movie or even (gasp) remaking it. And yet while hundreds of writers and filmmakers were inspired by “Blade Runner,” it’s hard to believe any of them could have found a way to expand its legacy as completely as Villenueve does here with a movie that doesn't feel at all repetitive. He’s in no way seeking to improve or replace—the films now work together, enriching each other instead of mimicking. They ask timeless questions and, like all great films, refuse to give you all the answers, allowing viewers to debate and discuss their meaning instead of merely being passive recipients of mindless entertainment. In that sense, "Blade Runner 2049" answers one of its own questions about what it means to be human—to have free thought—and how vital it is to appreciate art so clearly designed to enrich the soul. 

"""

In [58]:
review2 = '''
In the city of Ninjago, young Lloyd (voiced by Dave Franco) is scorned and bullied because his deadbeat dad is the evil Lord Garmadon (Justin Theroux); nobody knows that whenever Garmadon attacks the city, it’s Lloyd, disguised as the Green Ninja, who leads the counterattack. When Garmadon conquers the city, Green Ninja and his band embark on a quest to retake the city—and bridge the chasm between father and son. The LEGO movie franchise may be reaching a point of diminishing returns. Cranked out by an unruly mob of three directors and nine writers, this third outing is of interest only to kids ages five to seven—and any parents with serious daddy issues. MVP goes to Jackie Chan, who introduces the story in a live-action framing sequence, voices Lloyd’s guru, and choreographs the LEGO martial arts fights. J.L.

'''

In [65]:
review3 = '''
American Assassin is as generic an espionage thriller as you’re likely to find. Leaving no cliché unturned and offering a roster of predictable “twists” and unsurprising “surprises”, the production seems more like it was assembled from the leftover bits and pieces of better movies (most of them featuring James Bond) than the result of a screenplay written by four Hollywood veterans, one of whom is an Oscar winner. The story is so obvious that a viewer could leave the theater for fifteen minutes and not be even a little lost upon his return.

The opening scenes sets up the scenario with remarkable economy. Mitch Rapp (Dylan O’Brien) is on vacation in Spain with his girlfriend, Katrina (Charlotte Vega). He proposes marriage and she accepts. Moments later, she is dead, the victim of a terrorist attack. The time frame skips ahead 18 months. Mitch, having devoted his life to revenge, has infiltrated the cell responsible for his girlfriend’s death. At his first face-to-face meeting with the extremists, as he is about to make his move, the CIA intervenes and usurps Mitch’s revenge, killing the terrorists. He is then offered a job by Irene Kennedy (Sanaa Lathan), the director of one of the agency’s elite covert groups. In order to join, however, he must “pass” the brutal form of basic training presided over by the group’s leader, ex-Navy SEAL and all-around badass Stan Hurley (Michael Keaton).

After that, the movie is comprised of standard material involving an ex-agent (Taylor Kitsch) who wants revenge, Iranians trying to get their hands on the components necessary to build a nuclear bomb, a beautiful agent (Shiva Negar) whose personal pain rivals Mitch’s, and a brutal torture scene involving fingernails being ripped out. The movie has a distinct direct-to-video vibe to it but I guess that too many high-profile names were attached for it to be accorded that fate. The mere fact that it has been released theatrically doesn’t make it a better movie.

Director Michael Cuesta keeps the tone dark and dour. It’s strange for such a by-the-numbers action film to take itself so seriously. The filmmakers of John Wick and Atomic Blonde recognized that black humor and ostentatious, high-octane action scenes go a long way toward making something like this fun. Neither quality is in evidence, however. The screenplay attempts to say something about the pernicious effects of revenge on the soul, but the message gets muddled. The movie has a chance to redeem itself at the end and, for the briefest of moments, I thought it was going to do something daring…but it chickens out and plays it safe.

If there’s a reason to see this movie, it’s Michael Keaton, who invests his portrayal of Hurley with a callous, no-nonsense attitude that makes him interesting. Keaton has energy – something sorely lacking from the rest of the cast. The lead, The Maze Runner’s Dylan O’Brian is trapped in a permanent funk. Not only does he lack charisma but he mopes around so much it’s hard to root for him. Taylor Kitsch’s villain, who goes by the codename of “Ghost”, is an entitled whiner who’s peeved that Hurley didn’t fill his Daddy void. We’ve seen this character countless times before and Kitsch doesn’t bring anything to the part to distinguish him from his numerous predecessors.

There’s not much more to say about American Assassin. It’s a soulless example of a product assembled and packaged for those who crave an “old-fashioned” no-frills action movie. The film is made with sufficient competence not to be embarrassing but, while this isn’t a bottom-feeder, there are better options out there for aficionados of the genre. American Assassin kills time but accomplishes little else.

'''


In [68]:
print('The predict_proba is: {}'.format(loaded_gs.predict_proba([review1])))
print('The review is {}'.format(loaded_gs.predict([review1])[0]))

The predict_proba is: [[ 0.00570914  0.99429086]]
The review is POSITIVE
